In [6]:
import pickle
import string
import annoy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

In [7]:
checks = pd.read_csv("data/data.csv")
checks.drop(['name'], axis=1, inplace=True)

with open("data/Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)

checks.head()

C:\Users\saakyan.ag\AppData\Local\Temp\ipykernel_23944\4135959788.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  checks = pd.read_csv("data/data.csv")


,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00"


In [8]:
# Создадим ключевую колонку: время + касса + магазин
checks['key_col'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00",2018-12-07_1810323_1501.0


In [9]:
checks.dropna(inplace=True)
checks = checks[checks['sale_date_date'].str.startswith("20")]
checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)

# Достаточно отсортировать и сделать трейн-тест, чтобы построить валидацию. Забьем на разорванные сессии
train, test = train_test_split(checks, test_size=0.3)

In [25]:
# Обучим простые контетные рекомендации

# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

sentences = [preprocess_txt(k) for k in product_dict.keys()]

# Подготовим кандидатогенератор, который будет отдавать фильмы похожие по текстовому описанию на те, .
# которые оенил пользователь
# Обучим Fasttext и заэмбедим фильмы
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)


In [27]:

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')

# Будем хранить соответствия не только id-> фильм, но и фильм-> id, чтобы быстрее находить эмбеддинги айтемов
reverse_index_map = {}
counter = 0

for name in tqdm_notebook(product_dict.keys()):
    n_ft = 0
    reverse_index_map[counter] = name
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in preprocess_txt(name):
        # if word in modelFT:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_names.add_item(counter, vector_ft)
    counter += 1

#
ft_index_names.build(10)

C:\Users\saakyan.ag\AppData\Local\Temp\ipykernel_23944\1230621081.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm_notebook(product_dict.keys()):


  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [28]:
reverse_item_mapping = {v: k for k, v in product_dict.items()}

def recommend(items_list):
    current_vector = np.zeros(20)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in reverse_index_map:
            c -= 1
            continue
        current_vector += np.array(ft_index_names.get_item_vector[reverse_index_map[iname]])
    return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [29]:
recommend(['52539', '110342'])

C:\Users\saakyan.ag\AppData\Local\Temp\ipykernel_23944\3531675664.py:12: RuntimeWarning: invalid value encountered in divide
  return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]


['(106357) Салфетки EURON д/ухода за лежачими больными №64 755',
 '(79205) Верапамил тб п/о 80мг N30 738',
 '(5061) Лейкопластырь 2см х 500см катушка в уп.из полист. N1 670',
 '(64790) Ксимелин ЭКО с ментолом спрей назальный 140мкг/доза 1мг/мл фл.10мл 701',
 '(194632) Корега крем для фиксации зубных протезов Максимальное прилегание 40г 533',
 '(63827) Аркоксиа табл. п.п.о.90мг №7 647',
 '(91731) Селенцин шампунь от выпадения волос 200мл 476',
 '(112926) Либридерм Пантенол Форте 9% крем 30мл 547',
 '(91616) Гевискон Двойное Действие сусп.д/приема внутрь (мятная) пакет 10мл №12 595',
 '(196468) СалиЦинк Пенка для умывания для жирной и комбинированной кожи Сера-Цинк 160мл 503']

In [30]:
# Обучение эмбеддингов по последовательности
from gensim.models import Word2Vec

train['product_id'] = train['product_id'].apply(int).apply(str)

grouped = train.groupby('key_col')
sentences = []

In [31]:
# Сформируем последовательности

from tqdm import tqdm
sentences = []
for group in tqdm_notebook(grouped.groups):
    products = grouped.get_group(group)['product_id'].values
    if len(products) < 3:
        continue
    sentences.append(list(products))

C:\Users\saakyan.ag\AppData\Local\Temp\ipykernel_23944\4185597927.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for group in tqdm_notebook(grouped.groups):


  0%|          | 0/5458610 [00:00<?, ?it/s]

In [33]:
modelW2V = Word2Vec(sentences, vector_size=10)

In [34]:
sentences[0]

['69028', '62087', '120171']

In [56]:
def recommend_w2v(items_list):
    current_vector = np.zeros(10)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        # if iname not in modelW2V:
        #     c -= 1
        #     continue
        try:
            current_vector += np.array(modelW2V.wv[reverse_index_map[iname]])
        except KeyError:
            continue
    return [product_dict[i[0]] for i in modelW2V.wv.most_similar(current_vector / c, 10)]

In [57]:
recommend_w2v(['99821', '138583', '45321', '134475'])

['(91831) Солгар Тройная Омега-3 950 ЭПК и ДГК капсулы №50 банка 428',
 '(49371) Ципралекс таб. п.о.10мг №28 635',
 '(95819) Стрезам капс 50мг №60 631',
 '(66287) Селектра табл. п.п.о 10мг №28 635',
 '(43055) Золофт табл. п.о. 50 мг. №28 635',
 '(55087) Адаптол таб.500мг №20 632',
 '(114206) Эглонил капс 50мг N30 637',
 '(66484) Солгар Коэнзим Q-10 капс. 60мг №30 428',
 '(112174) Нэйчес Баунти Рыбий Жир Омега-3 капс.980мг №30 439',
 '(72189) Витамин Е Зентива капс.400 мг №30 438']